In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
dataset = pd.read_csv(r"./Datasets/FTSE/FTSE_MFDFA.csv")
df = dataset.reset_index(drop=True)
df

,Date,Open,High,Low,Close,Hurst
0,2011/1/6,2704.38,2712.35,2697.73,2709.89,0.584601
1,2011/1/7,2712.75,2715.96,2676.36,2703.17,0.594511
2,2011/1/10,2691.47,2712.32,2682.25,2707.80,0.601399
3,2011/1/11,2719.60,2722.75,2706.58,2716.83,0.598036
4,2011/1/12,2731.45,2737.33,2722.32,2737.33,0.594976
...,...,...,...,...,...,...
1751,2017/12/20,6991.26,6991.26,6935.42,6960.96,0.407652
1752,2017/12/21,6972.88,6992.63,6961.20,6965.36,0.397472
1753,2017/12/22,6958.02,6962.26,6944.44,6959.96,0.411361
1754,2017/12/26,6928.92,6942.14,6915.56,6936.25,0.405433


In [2]:
maxLevel = 21                           
maxTP = 1                
maxTS = 4096             
nTP = 0                    
p3 = 1.0 / 3.0          

stime = 0
etime = 0
Gstime = 0
Getime = 0
tlapse = 0
Gtlapse = 0

day = 2261

DT_DT= np.zeros((day,1))

DT_OP= np.zeros((day,1))
DT_HI= np.zeros((day,1))
DT_LO= np.zeros((day,1))
DT_CL= np.zeros((day,1))
DT_VL= np.zeros((day,1))

cHurst= np.zeros((day,1))
oHurst= np.zeros((day,1))
hHurst= np.zeros((day,1))
lHurst= np.zeros((day,1))
DT_RT= np.zeros((day,1))

Q= np.zeros((100,1))
NQ= np.zeros((100,1))
r= np.zeros((100,1))


QFEL = np.zeros((21,1))       
QPR = np.zeros((21,1))       
NQPR = np.zeros((21,1))      
K = np.zeros((21,1))


ALL_Pos_QPL = np.zeros((day,21))
ALL_Neg_QPL = np.zeros((day,21))

print("Printout ALL K values K0 .. K20 for first 20 Levels")
for L in range(0,21):    
    K[L] = math.pow((1.1924 + (33.2383 * L) + (56.2169 * L * L)) / (1 + (43.6106 * L)), p3)
    print("Level ",L, " K",L, " = ", K[L])   

Printout ALL K values K0 .. K20 for first 20 Levels
Level  0  K 0  =  [1.06041043]
Level  1  K 1  =  [1.2665998]
Level  2  K 2  =  [1.4911995]
Level  3  K 3  =  [1.66349934]
Level  4  K 4  =  [1.80609901]
Level  5  K 5  =  [1.92919156]
Level  6  K 6  =  [2.03832293]
Level  7  K 7  =  [2.13688129]
Level  8  K 8  =  [2.22710524]
Level  9  K 9  =  [2.31055991]
Level  10  K 10  =  [2.38838747]
Level  11  K 11  =  [2.46144992]
Level  12  K 12  =  [2.53041565]
Level  13  K 13  =  [2.59581465]
Level  14  K 14  =  [2.65807505]
Level  15  K 15  =  [2.71754825]
Level  16  K 16  =  [2.77452666]
Level  17  K 17  =  [2.82925649]
Level  18  K 18  =  [2.88194724]
Level  19  K 19  =  [2.93277882]
Level  20  K 20  =  [2.98190702]


In [3]:
DT_DT = df['Date']     
DT_OP = df['Open']
DT_HI = df['High']
DT_LO = df['Low']
DT_CL = df['Close']
hurst = df['Hurst']
TSsize = df.shape[0]

TSsize

1756

In [4]:
# For Hurst
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_CL[d + 1] > 0):
        DT_RT[d] = DT_CL[d] / DT_CL[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurst[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurst[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/FTSE/FTSE_MFDFA+qhl.csv",index = True)

In [6]:
qplp21 = cQHLdata["cQHLp21"]
qpln21 = cQHLdata["cQHLn21"]

plt.figure(figsize=(24, 16), dpi=400)
plt.plot(qplp21,color='aquamarine')
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlabel('Days', fontsize=35)
plt.ylabel('Energy Level', fontsize=35)
plt.title('FTSE_MFDFA_QHLp21', fontsize=40)
plt.legend(['Energy Level'],fontsize=40)
plt.savefig('./pic/FTSE_pic/FTSE_MFDFA_QHLp21.png')
plt.show()

plt.figure(figsize=(24, 16), dpi=400)
plt.plot(qpln21,color='deepskyblue')
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlabel('Days', fontsize=35)
plt.ylabel('Energy Level', fontsize=35)
plt.title('FTSE_MFDFA_QHLn21', fontsize=40)
plt.legend(['Energy Level'],fontsize=40)
plt.savefig('./pic/FTSE_pic/FTSE_MFDFA_QHLn21.png')
plt.show()